<div align="center" dir="rtl">
<h2>تمرین چهارم درس بازیابی پیشرفته‌ی اطلاعات</h2>
<h3>گروه سی‌و‌سوم: علی مهربانی، پانیذ حلواچی، آرمان سلیمانی</h3>
</div>
<div align="right" dir="rtl">
  <br>
    در این تمرین، تسک دسته‌بندی روی داده‌های گرد‌آوری شده از بوستان سعدی انجام می‌شود. ابتدا دیتاست مورد نیازمان از روی داده‌های خام که برای تمرین سوم جمع شده بود ساخته می‌شود، سپس قسمت پیش‌پردازش قرار دارد و در ادامه تسک روی داده‌ها انجام می‌شود.
</div>

<div align="right" dir="rtl">
    <h3>قسمت اول: آماده‌سازی دیتاست</h3>
  <br>
    در این قسمت فایل‌های داده‌های خام خوانده شده و دیتاست بیت به بیت برای کلاس‌بندی ایجاد می‌شود.
</div>

In [2]:
#import libraries for dataset generation
import pandas as pd
from os import listdir
!pip install hazm
!pip install torchtext
from hazm import *
import torch
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import itertools

You should consider upgrading via the 'c:\users\sky\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\sky\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [3]:
all_files = listdir('Boostan/')

i = 0
df = pd.DataFrame(columns=['beyt', 'baab'])

for p in all_files:
    f = open('Boostan/' + p, 'r', encoding='utf8')
    
    has_poems = True
    
    while has_poems:
        poem = ''
        
        while True:
            s = f.readline()
            if '$' in s:
                has_poems = False
                break
            elif '_' in s:
                break
            
            #age injaiim yani s ye mesra boode
            beyt = s + f.readline()[:-1]
            df.loc[i] = [beyt, p[:-4]]
            i = i + 1

In [4]:
print(df.at[2000, 'beyt'])
print(df.at[2000, 'baab'])
print(df.shape[0])

نپنداری این قول معقول نیست
چو قانع شدی سیم و سنگت یکی است
باب ششم در قناعت
3845


<div align="right" dir="rtl">
    <h3>قسمت دوم: پیش‌پردازش</h3>
  <br>
    در این قسمت یک نمونه‌ی پیش‌پردازش شده از دیتاست تهیه می‌شود. دیتاست پیش‌پردازش‌نشده هم در کنار آن حفظ می‌شود که امکان مقایسه‌ی نتیجه روی دو دیتاست وجود داشته باشد.
    توضیحات دقیق این که هر گام از پیش‌پردازش شامل چه مواردی است، در تمرین سوم آورده شده است.
</div>

In [5]:
#import libraries for preprocessing
from __future__ import unicode_literals
from hazm import *
import json
import codecs

In [6]:
df_raw = df.copy()

<div align="right" dir="rtl">
    <h4>مرحله‌ی اول پیش‌پردازش</h4>
  <br>
    در این گام، برخی موارد در واژگان (مانند «گه» که همان «گاه» است) اصلاح می‌شوند. عملیات نرمالایز کردن و لمتایز کردن هم انجام داده می‌شود.
</div>

In [7]:
normalizer = Normalizer()
lemmatizer = Lemmatizer()

def preprocess_1(beyt):
    beyt = normalizer.normalize(beyt).replace(' ز ', ' از ').replace(' مر ', ' ') #normalize beshe va z beshe az
    
    words = word_tokenize(beyt)
    
    for j in range(0, len(words)):
        
        word = words[j]
        past_root = lemmatizer.lemmatize(word).split('#', 1)[0]
        past_root = past_root.replace('آ', 'ا')
        word_modified = word.replace(past_root, '')
        
        if word_modified != '':
            if word_modified[0] == 'ب' and len(word_modified) < 4:
                #yani fele maazi bode be forme ghadimi
                beyt = beyt.replace(word, word[1:])
        
        if word.endswith('گه'):
            beyt = beyt.replace(word, word[:-2]+'گاه')
        
        return beyt

for i in range(0, df.shape[0]):
    df.at[i, 'beyt'] = preprocess_1(df.at[i, 'beyt'])

<div align="right" dir="rtl">
    <h4>مرحله‌ی دوم پیش‌پردازش</h4>
  <br>
    در این گام، حالات باستانی افعال به حالت عادی تبدیل می‌شود تا افعال یک دست باشند.
</div>

In [8]:
def preprocess_2(beyt):
    words = word_tokenize(beyt)
    
    for j in range(0, len(words)):
        
        word = words[j]
        
        if word[0] == 'م':
            word_modified = 'ب' + word[1:]
            present_root = lemmatizer.lemmatize(word_modified).split('#', 1)[-1]
            present_root = present_root.replace('آ', 'ا')
            word_modified = word_modified.replace(present_root, '')
            
            if word_modified == 'ب':
                beyt = beyt.replace(word, 'ن'+word[1:])
    return beyt

for i in range(0, df.shape[0]):            
    df.at[i, 'beyt'] = preprocess_2(df.at[i, 'beyt'])

<div align="right" dir="rtl">
    <h4>مرحله‌ی سوم پیش‌پردازش</h4>
  <br>
    در این مرحله، استاپ ووردها حذف شده و برخی واژگان خاص پیش‌پردازش می‌شوند.
</div>

In [9]:
with open('Exception.txt', encoding='utf8') as exc:
    data = exc.read()
special_verbs = json.loads(data)

df_temp = pd.DataFrame(columns=['beyt', 'baab'])
stopwords = ['!','،','؟','ز','ار']+[normalizer.normalize(x.strip()) for x in codecs.open('stopwords.txt','r','utf-8').readlines()]


def preprocess_3(beyt):
    words = word_tokenize(beyt)
    words = [t for t in words if t not in stopwords]
    lemmatized_tokens = []
    
    for t in words:
        if t in special_verbs.keys():
            lemmatized_tokens.append(t)
        else:
            lemmatized_tokens.append(lemmatizer.lemmatize(t).split('#')[0])
    
    lemmatized = ' '.join(lemmatized_tokens)
            
    return lemmatized, lemmatized_tokens

for i in range(0, df.shape[0]):
    pp = preprocess_3(df.at[i, 'beyt'])
            
    df.at[i, 'beyt'] = pp[0]
    df_temp.at[i, 'beyt'] = pp[1]


In [10]:
print(df.at[2000, 'beyt'])
print(df.at[2000, 'baab'])

نپنداری قول معقول چو قانع شد سیم سنگ
باب ششم در قناعت


In [11]:
#test preprocessing
preprocess_3(preprocess_2(preprocess_1('''نپنداری این قول معقول نیست
چو قانع شدی سیم و سنگت یکی است''')))[0]

'نپنداری قول معقول چو قانع شد سیم سنگ'

<div align="right" dir="rtl">
    <h4>تابع پیش‌پردازش تک‌بیت</h4>
  <br>
    حال یک تابع درست می‌کنیم که یک بیت را پیش‌پردازش کند تا بعدا هنگام دریافت ورودی سامانه از آن استفاده کنیم.
</div>

In [12]:
def preprocess(beyt):
    return preprocess_3(preprocess_2(preprocess_1(beyt)))[0]

In [13]:
#test preprocessing functions
print(df_raw.at[2000, 'beyt'] + '\n----')
print(df.at[2000, 'beyt'] + '\n----')
print(preprocess(df_raw.at[2000, 'beyt']))

نپنداری این قول معقول نیست
چو قانع شدی سیم و سنگت یکی است
----
نپنداری قول معقول چو قانع شد سیم سنگ
----
نپنداری قول معقول چو قانع شد سیم سنگ


<div align="right" dir="rtl">
    <h3>قسمت سوم: کلاس‌بندی با scikit</h3>
  <br>
</div>

<div align=center dir='rtl'>
		    <font size=4 color=green>
			    <br />
                مدل اول:
                </font>
                <font size=3>
                Logistic Regression (+ Isolation Forest)
            	<br/>
			</font>
    <br/>
    </div>
    <hr/>

In [14]:
#import libraries for classification (1)
from sklearn.ensemble import IsolationForest
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import shuffle
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
import numpy as np

In [15]:
vectorizer = TfidfVectorizer(ngram_range=(1, 1), stop_words=None, norm='l2')

In [16]:
# prepare
df = shuffle(df)
df.reset_index(inplace=True, drop=True)
X_text = [x.strip().split() for x in df.beyt]
X_vec = vectorizer.fit_transform([' '.join(x) for x in X_text])
y = [x for x in df.baab]

In [17]:
X_vec

<3845x4886 sparse matrix of type '<class 'numpy.float64'>'
	with 27936 stored elements in Compressed Sparse Row format>

In [18]:
def make_logreg_model(x_in, y_in):
    model_1 = LogisticRegression(random_state=0, max_iter=200)
    cv_results = cross_validate(model_1, x_in, y_in, cv=5, return_estimator=True, scoring='f1_macro')
    print(cv_results['test_score'])

In [19]:
make_logreg_model(X_vec, y)

[0.15926461 0.17260103 0.169607   0.18972463 0.19634209]


<div align="right" dir="rtl">
    <h4>بررسی آوگمنتیشن دیتاست</h4>
  <br>
    با توجه به دقت کم، در این قسمت بررسی می‌کنیم که افزایش مصنوعی دیتاست چه کمکی ممکن است به ما بکند.
</div>

In [20]:
X_noise = np.random.normal(0, 0.1, X_vec.shape)
X_noisy_duplicate = X_noise + X_vec.toarray()
X_augmented = np.vstack((X_vec.toarray(), X_noisy_duplicate))
y_augmented = y*2

In [21]:
make_logreg_model(X_augmented, y_augmented)

[0.15901636 0.15865    0.19508017 0.17765603 0.20991097]


<div align="right" dir="rtl">
  <br>
    بنابراین مشاهده می‌شود که با دو برابر کردن دیتاست، بهترین جواب موجود، تا حدی بهتر است و امتیاز f1
    از حدود ۰.۱۷۷ برای بهترین مدل، به ۰.۲۰۲ برای بهترین مدل رسید. اما هنوز هم بخاطر نزدیک بودن داده‌ها، این امتیاز چنگی به دل نمی‌زند.
</div>

<div align="right" dir="rtl">
    <h4>تحلیل خروجی‌های یک مدل</h4>
  <br>
    حال که حدود دقت برای این نوع مدل را می‌دانیم، یک مدل می‌سازیم و متریک‌های خواسته شده و فیچرها را مشاهده می‌کنیم.
</div>

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X_augmented, y_augmented, test_size=0.15, random_state=1)

In [23]:
model_1 = LogisticRegression(random_state=0, max_iter=200).fit(X_train, y_train)

In [24]:
y_inf = model_1.predict(X_test)
f1_score(y_test, y_inf, average='macro')

0.17705612549388353

In [25]:
accuracy_score(y_test, y_inf)

0.26603119584055457

In [26]:
confusion_matrix(y_test, y_inf)

array([[155,   0,  17,  11,   4,  15,   7,  17,   2,  21],
       [ 14,   0,   5,   1,   0,   3,   0,   3,   0,   6],
       [ 68,   0,  35,  10,   0,   2,   2,  16,   3,  18],
       [ 37,   1,  12,  20,   0,   8,   4,   9,   3,  16],
       [ 19,   0,  11,   4,   3,   1,   3,   5,   3,   7],
       [ 36,   0,  14,   5,   1,  13,   6,   8,   1,  10],
       [ 41,   0,   2,   6,   0,   6,   5,   7,   0,  15],
       [ 69,   1,  12,   6,   4,   5,   2,  26,   2,  13],
       [ 25,   1,   8,   4,   0,   3,   1,   6,   4,   7],
       [ 74,   2,  15,   7,   2,  10,   2,  17,   3,  46]], dtype=int64)

<div align="right" dir="rtl">
    <h4>فیچرها</h4>
  <br>
    مشاهده می‌کنیم که چه فیچرهایی انتخاب شده‌اند.
</div>

In [27]:
[vectorizer.get_feature_names()[i] for i in model_1.coef_[0].argsort()[-10:][::-1]]

c:\users\sky\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['ملک', 'عدو', 'دشمن', 'لشکر', 'ظالم', 'سپاه', 'رعیت', 'جنگ', 'کشور', 'گزند']

In [28]:
vectorizer.get_feature_names()

['آب',
 'آباد',
 'آبدان',
 'آبرو',
 'آبستن',
 'آبکش',
 'آبگینه',
 'آتش',
 'آتشم',
 'آتشین',
 'آثار',
 'آختن',
 'آخته',
 'آخر',
 'آخرت',
 'آداب',
 'آدم',
 'آدمی',
 'آذر',
 'آذرپرست',
 'آذین',
 'آر',
 'آرا',
 'آراسته',
 'آرام',
 'آرایش',
 'آرد',
 'آرزومند',
 'آرش',
 'آرم',
 'آرمید',
 'آرند',
 'آری',
 'آز',
 'آزاد',
 'آزادمرد',
 'آزاده',
 'آزادگان',
 'آزادگی',
 'آزار',
 'آزرده',
 'آزمای',
 'آزمود',
 'آزمودم',
 'آزموده',
 'آزمودی',
 'آسان',
 'آساید',
 'آسایدش',
 'آسایش',
 'آستان',
 'آستر',
 'آستین',
 'آسمان',
 'آسود',
 'آسوده',
 'آسودگی',
 'آسیب',
 'آشتی',
 'آشفت',
 'آشفته',
 'آشفتگی',
 'آشنا',
 'آشنایی',
 'آشوب',
 'آشکار',
 'آغاز',
 'آغشته',
 'آغوش',
 'آفاق',
 'آفتاب',
 'آفرید',
 'آفریدت',
 'آفریدندت',
 'آفرین',
 'آفریند',
 'آفریننده',
 'آل',
 'آلایش',
 'آلایشم',
 'آلوده',
 'آماج',
 'آماجگاه',
 'آمد',
 'آمدت',
 'آمدش',
 'آمدن',
 'آمده',
 'آمرزگار',
 'آموخت',
 'آموخته',
 'آموز',
 'آموزگار',
 'آمیختستند',
 'آمیزگار',
 'آمیزگاری',
 'آن',
 'آنچ',
 'آنگه',
 'آنی',
 'آه',
 'آهخت',
 'آهخته',
 'آ

<div align="right" dir="rtl">
    <h4>بررسی outlier detection</h4>
  <br>
    سعی می‌کنیم با استفاده از isolation forest یک سیستم
    برای تشخیص اشعاری که مرتبط با سعدی نیستند داشته باشیم. 
    کاربرد این لایه برای زمانی است که بخواهیم یک بیت دلخواه را کلاس‌بندی کنیم که ممکن است از بوستان نباشد.
</div>

In [29]:
isfo = IsolationForest(random_state=0).fit(X_train)
isfo.predict(X_test[0:5])

array([1, 1, 1, 1, 1])

In [30]:
noisy_input = X_test[0:5] + np.random.normal(0, 0.1, X_test[0:5].shape)
isfo.predict(noisy_input)

array([-1, -1,  1, -1, -1])

In [31]:
random_input = np.random.normal(0, 0.5, X_test[0:5].shape)
isfo.predict(random_input)

array([-1, -1, -1, -1, -1])

<div align="right" dir="rtl">
  <br>
    بنابراین مشاهده می‌شود که عملکرد این مدل روی داده‌هایی که از بوستان داریم مناسب است و لااقل فرق بین ابیات نویزی و ابیات عادی تا حد خوبی مشخص است و فرق بیت داده‌ی رندوم و ابیات عادی خوب است.
</div>

<div align=center>
		    <font size=4 color=green>
			    <br />
                مدل دوم:
                </font>
                <font size=3>
                مدل مبتنی بر ترنسفرمر
            	<br/>
			</font>
    <br/>
    </div>
    <hr/>

In [32]:
from transformers import AutoConfig, AutoTokenizer, AutoModel, Trainer, TrainingArguments, AutoModelForSequenceClassification, pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import torch
#model: https://huggingface.co/HooshvareLab/bert-base-parsbert-uncased/tree/main

In [33]:
model_name = "HooshvareLab/bert-base-parsbert-uncased"
config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=10)

Some weights of the model checkpoint at HooshvareLab/bert-base-parsbert-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassifica

In [34]:
X = []
Y = []
labels_dict = {}
labels_dict['باب اول در عدل و تدبیر و رای'] = 0
labels_dict['باب دوم در احسان'] = 1
labels_dict['باب سوم در عشق و مستی و شور'] = 2
labels_dict['باب چهارم در تواضع'] = 3
labels_dict['باب پنجم در رضا'] = 4
labels_dict['باب ششم در قناعت'] = 5
labels_dict['باب هفتم در عالم تربیت'] = 6
labels_dict['باب هشتم در شکر بر عافیت'] = 7
labels_dict['باب نهم در توبه و راه صواب'] = 8
labels_dict['باب دهم در مناجات و ختم کتاب'] = 9
for i in range(0, df.shape[0]):
    X.append(df.at[i, 'beyt'])
    Y.append(labels_dict[df.at[i, 'baab'].strip()])
X_train, X_testdev, Y_train, Y_testdev = train_test_split(X, Y, test_size=0.2, random_state=1, shuffle=True)
X_test, X_dev, Y_test, Y_dev = train_test_split(X_testdev, Y_testdev, test_size=0.5, random_state=1, shuffle=True) 

In [35]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)
dev_encodings = tokenizer(X_dev, truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [36]:
class PoemDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = PoemDataset(train_encodings, Y_train)
test_dataset = PoemDataset(test_encodings, Y_test)
dev_dataset = PoemDataset(dev_encodings, Y_dev)

In [37]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=100,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset
)

trainer.train()

c:\users\sky\appdata\local\programs\python\python38\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3076
  Num Epochs = 10
  Instantaneous batch size per device = 100
  Total train batch size (w. parallel, distributed & accumulation) = 100
  Gradient Accumulation steps = 1
  Total optimization steps = 310


Step,Training Loss
10,2.314600
20,2.263800
30,2.225300
40,2.211300
50,2.162500
60,2.122700
70,2.156200
80,2.105300
90,2.117100
100,2.097100




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=310, training_loss=1.82855468257781, metrics={'train_runtime': 2724.4047, 'train_samples_per_second': 11.291, 'train_steps_per_second': 0.114, 'total_flos': 268742022860640.0, 'train_loss': 1.82855468257781, 'epoch': 10.0})

In [38]:
generator = pipeline('text-classification', model, tokenizer=tokenizer, config={'max_length':256})
Y_pred = [int(y['label'].split('_')[1]) for y in generator(X_test)]
f1_score(Y_test, Y_pred, average='macro')

Disabling tokenizer parallelism, we're using DataLoader multithreading already


0.17683485360544685

In [39]:
accuracy_score(Y_test, Y_pred)

0.2786458333333333

In [40]:
confusion_matrix(y_test, y_inf)

array([[155,   0,  17,  11,   4,  15,   7,  17,   2,  21],
       [ 14,   0,   5,   1,   0,   3,   0,   3,   0,   6],
       [ 68,   0,  35,  10,   0,   2,   2,  16,   3,  18],
       [ 37,   1,  12,  20,   0,   8,   4,   9,   3,  16],
       [ 19,   0,  11,   4,   3,   1,   3,   5,   3,   7],
       [ 36,   0,  14,   5,   1,  13,   6,   8,   1,  10],
       [ 41,   0,   2,   6,   0,   6,   5,   7,   0,  15],
       [ 69,   1,  12,   6,   4,   5,   2,  26,   2,  13],
       [ 25,   1,   8,   4,   0,   3,   1,   6,   4,   7],
       [ 74,   2,  15,   7,   2,  10,   2,  17,   3,  46]], dtype=int64)

<div align=center>
		    <font size=4 color=green>
			    <br />
                مدل سوم:
                </font>
                <font size=3>
                مدل شبکه عصبی استفاده از کتابخانه
                
torchtext
            	<br/>
			</font>
    <br/>
    </div>
    <hr/>

In [21]:
#import libraries for classification (1)
import os
import re
import shutil
import string
from collections import Counter
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from torch import nn
import torch.nn.functional as F
import time
from sklearn.metrics import f1_score
from sklearn import metrics
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
corresponding_label = {"باب اول در عدل و تدبیر و رای":1,"باب دوم در احسان":2,"باب سوم در عشق و مستی و شور":3,"باب چهارم در تواضع":4,"باب پنجم در رضا":5,"باب ششم در قناعت":6,"باب هفتم در عالم تربیت":7,"باب هشتم در شکر بر عافیت":8,"باب نهم در توبه و راه صواب":9,'باب دهم در مناجات و ختم کتاب ':10}

In [23]:
#vectorized data
vectorizer = TfidfVectorizer(ngram_range=(1, 1), stop_words=None, norm='l2')
v = vectorizer.fit_transform(['نپنداری قول معقول چو قانع شد سیم سنگ', 'ممد ممد', 'ممد'])
#v = augment(v)

In [24]:
#split the data to training data and validation 
X_train, X_valid, Y_train, Y_valid= train_test_split(df['beyt'].tolist(),\
                                                      df['baab'].tolist(),\
                                                      test_size=0.2,\
                                                      stratify = df['baab'].tolist(),\
                                                      random_state=3)

train_dat =list(zip(Y_train,X_train))
valid_dat =list(zip(Y_valid,X_valid))

In [25]:
#define the voceb
train_iter = train_dat
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield word_tokenize(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [26]:
#define label and data piplines
text_pipeline = lambda x: vocab(word_tokenize(x))
label_pipeline = lambda x: corresponding_label[x]-1

In [27]:
#one layer model
class TextClassificationModel1(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel1, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

#two-layer model
class TextClassificationModel2(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel2, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc1 = nn.Linear(embed_dim,16)
        self.fc2 = nn.Linear(16,num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc1.weight.data.uniform_(-initrange, initrange)
        self.fc1.bias.data.zero_()
        self.fc2.weight.data.uniform_(-initrange, initrange)
        self.fc2.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        x = F.relu(self.fc1(embedded))
        x = self.fc2(x)
        return x

#three-layer model
class TextClassificationModel3(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel3, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc1 = nn.Linear(embed_dim,16)
        self.fc2 = nn.Linear(16,num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc1.weight.data.uniform_(-initrange, initrange)
        self.fc1.bias.data.zero_()
        self.fc2.weight.data.uniform_(-initrange, initrange)
        self.fc2.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        x = F.relu(self.fc1(embedded))
        x = self.fc2(x)
        return x

In [28]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

In [29]:
#embedding size is a hyperparameter which we should tune it
emsize = 32
train_iter1 = train_dat
num_class = len(set([label for (label, text) in train_iter1]))
vocab_size = len(vocab)
model = TextClassificationModel1(vocab_size, emsize, num_class).to(device)

In [30]:
#train the data
def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 50
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            # print('| epoch {:3d} | {:5d}/{:5d} batches '
            #       '| accuracy {:8.3f}| loss {:8.3f}'.format(epoch, idx, len(dataloader),
            #                                   total_acc/total_count, loss.item()))
            total_acc, total_count = 0, 0
            start_time = time.time()

#evaluate the validation data
def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0
    y_true=[]
    y_pred=[]
    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            y_pred.append(predicted_label.argmax(1).tolist())
            y_true.append(label.tolist())
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    print(metrics.confusion_matrix(list(itertools.chain(*y_true)), list(itertools.chain(* y_pred)), labels=[0,1,2,3,4,5,6,7,8,9]))
    return total_acc/total_count, f1_score(list(itertools.chain(*y_true)),list(itertools.chain(* y_pred)), average='macro')

In [34]:
# Hyperparameters
EPOCHS = 20 # epoch
LR = 10 # learning rate
BATCH_SIZE = 64 # batch size for training

In [35]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

train_iter2 = train_dat
valid_iter2= valid_dat


train_dataloader = DataLoader(train_iter2, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(valid_iter2, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)

In [36]:
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val,f_macro = evaluate(valid_dataloader)

    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} | f1_macro: {:8.3f}  '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val,f_macro))
    print('-' * 59)

[[89 26  8 25  2  1 17 10  4  0]
 [22 35  2 21  1  1  6  7  4  1]
 [18 14 13 15  0  0  8  0  5  0]
 [23 19  6 32  0  3 17  1  4  0]
 [11  7  4  7  0  1  6  1  3  1]
 [10  7  0  8  0  3  3  2  0  1]
 [29 13  4 11  0  3 21  3  5  0]
 [11 11  2 17  2  0  7  3  2  0]
 [18  9  2 15  0  1  5  2 15  0]
 [ 4  2  0  8  0  1  5  2  1  0]]
-----------------------------------------------------------
| end of epoch   1 | time:  0.18s | valid accuracy    0.274 | f1_macro:    0.188  
-----------------------------------------------------------
[[107  22   8  22   2   1  13   3   4   0]
 [ 30  29   3  23   0   1   7   3   3   1]
 [ 21  13  16  15   0   0   6   0   2   0]
 [ 29  12   6  36   0   3  15   1   3   0]
 [ 12   6   5   8   0   1   4   1   3   1]
 [ 10   6   0   8   0   4   3   0   1   2]
 [ 32  11   4  13   0   3  19   3   4   0]
 [ 18  11   1  16   2   0   4   2   1   0]
 [ 22   7   2  13   0   1   5   2  15   0]
 [  7   2   0   7   0   1   4   1   1   0]]
-----------------------------------

[[89 22 11 28  3  3  9 11  6  0]
 [18 27  5 27  1  1  6  7  7  1]
 [14  7 22 16  0  1  6  0  7  0]
 [21 10  8 38  2  6 15  2  3  0]
 [10  5  5  8  0  2  6  1  3  1]
 [ 8  6  0  7  0  6  3  2  1  1]
 [28 11  4 16  0  3 19  3  5  0]
 [14  6  2 17  3  0  6  3  4  0]
 [20  5  2 17  1  1  3  3 15  0]
 [ 4  2  0  9  0  1  4  2  1  0]]
-----------------------------------------------------------
| end of epoch  18 | time:  0.18s | valid accuracy    0.285 | f1_macro:    0.206  
-----------------------------------------------------------
[[89 22 11 28  3  3  9 11  6  0]
 [18 27  5 27  1  1  6  7  7  1]
 [14  7 22 16  0  1  6  0  7  0]
 [21 10  8 38  2  6 15  2  3  0]
 [10  5  5  8  0  2  6  1  3  1]
 [ 8  6  0  7  0  6  3  2  1  1]
 [28 11  4 16  0  3 19  3  5  0]
 [14  6  2 17  3  0  6  3  4  0]
 [20  5  2 17  1  1  3  3 15  0]
 [ 4  2  0  9  0  1  4  2  1  0]]
-----------------------------------------------------------
| end of epoch  19 | time:  0.17s | valid accuracy    0.285 | f1_macro:    